In [1]:
# https://github.com/MhLiao/DB
# https://github.com/zonasw/DBNet
# https://github.com/xuannianz/DifferentiableBinarization
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
APPROACH_NAME = 'DBNet'

# Check GPU working

In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0': raise SystemError('GPU device not found')
print('Found GPU at:', device_name)
!nvcc -V

Found GPU at: /device:GPU:0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Nov_30_19:15:10_Pacific_Standard_Time_2020
Cuda compilation tools, release 11.2, V11.2.67
Build cuda_11.2.r11.2/compiler.29373293_0


# Data input pipeline

In [3]:
BATCH_SIZE = 4
IMAGE_SIZE = 640
THRESH_MIN = 0.3
THRESH_MAX = 0.7
SHRINK_RATIO = 0.4

In [4]:
from loader import DataImporter, DBNetDataGenerator
dataset = DataImporter('Datasets', pattern='*.cach')
print(dataset)

Samples count (1 image can have multiple bounding boxes):
- Number of images found: 100
- Number of image bounding boxes: 100
- Number of bounding boxes in all images: 2506


In [5]:
train_img_paths, all_train_bboxes, valid_img_paths, all_valid_bboxes = dataset.split(0.8)
print('Number of training samples:', len(train_img_paths))
print('Number of validate samples:', len(valid_img_paths))

Number of training samples: 80
Number of validate samples: 20


In [6]:
train_generator = DBNetDataGenerator(
    train_img_paths, all_train_bboxes, BATCH_SIZE, IMAGE_SIZE, 
    THRESH_MIN, THRESH_MAX, SHRINK_RATIO
)
valid_generator = DBNetDataGenerator(
    valid_img_paths, all_valid_bboxes, BATCH_SIZE, IMAGE_SIZE, 
    THRESH_MIN, THRESH_MAX, SHRINK_RATIO, False
)

# Define the model

In [7]:
from tensorflow.keras.layers import (
    Input, Convolution2D, Conv2DTranspose, UpSampling2D,
    BatchNormalization, Activation, Add, Concatenate, Lambda
)
from layers import ConvBnRelu, DeconvolutionalMap
from keras_resnet.models import ResNet50
from losses import db_loss

In [8]:
def DBNet(k=50, is_training=True, name='DBNet'):
    image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name='image')
    backbone = ResNet50(inputs=image_input, include_top=False, freeze_bn=True)
    
    C2, C3, C4, C5 = backbone.outputs
    in2 = ConvBnRelu(256, kernel_size=1, name=f'in2')(C2)
    in3 = ConvBnRelu(256, kernel_size=1, name=f'in3')(C3)
    in4 = ConvBnRelu(256, kernel_size=1, name=f'in4')(C4)
    in5 = ConvBnRelu(256, kernel_size=1, name=f'in5')(C5)
        
    P5 = ConvBnRelu(64, kernel_size=3, name='P5_conv')(in5)
    P5 = UpSampling2D(8, name='P5_up')(P5) # 1 / 32 * 8 = 1 / 4
    
    out4 = Add(name='out4')([in4, UpSampling2D(2, name='in5_up')(in5)])
    P4 = ConvBnRelu(64, kernel_size=3, name='P4_conv')(out4)
    P4 = UpSampling2D(4, name='P4_up')(P4) # 1 / 16 * 4 = 1 / 4
    
    out3 = Add(name='out3')([in3, UpSampling2D(2, name='out4_up')(out4)])
    P3 = ConvBnRelu(64, kernel_size=3, name='P3_conv')(out3)
    P3 = UpSampling2D(2, name='P3_up')(P3) # 1 / 8 * 2 = 1 / 4
    
    out2 = Add(name='out2')([in2, UpSampling2D(2, name='out3_up')(out3)])
    P2 = ConvBnRelu(64, kernel_size=3, name='P2_conv')(out2) # 1 / 4
    
    fuse = Concatenate(name='fuse')([P2, P3, P4, P5]) # (batch_size, /4, /4, 256)
    binarize_map = DeconvolutionalMap(64, name='probability_map')(fuse)
    if not is_training: return tf.keras.Model(inputs=image_input, outputs=binarize_map, name=name)
    threshold_map = DeconvolutionalMap(64, name='threshold_map')(fuse)
    
    gt_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE), name='gt_input')
    mask_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE), name='mask_input')
    thresh_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE), name='thresh_input')
    thresh_mask_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE), name='thresh_mask_input')
    
    # Approximate binary map
    b_hat = Lambda( 
        function = lambda x: 1 / (1 + tf.exp(-k * (x[0] - x[1]))), # 1 / (1 + e^(-k(P - T)))
        name = 'approximate_binary_map'
    )([binarize_map, threshold_map]) 
    
    loss_layer = Lambda(db_loss, name='db_loss')([
        gt_input, mask_input, thresh_input, thresh_mask_input, 
        binarize_map, b_hat, threshold_map
    ])

    model = tf.keras.Model(
        inputs = [image_input, gt_input, mask_input, thresh_input, thresh_mask_input], 
        outputs = [loss_layer],
        name = name
    )
    model.add_loss(model.get_layer('db_loss').output)
    return model

In [9]:
model = DBNet(is_training=True)
model.summary(line_length=120)

Model: "DBNet"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
image (InputLayer)                     [(None, 640, 640, 3)]      0                                                     
________________________________________________________________________________________________________________________
conv1 (Conv2D)                         (None, 320, 320, 64)       9408          image[0][0]                             
________________________________________________________________________________________________________________________
bn_conv1 (BatchNormalization)          (None, 320, 320, 64)       256           conv1[0][0]                             
________________________________________________________________________________________________________________________
conv1_relu (Activ

# Callbacks

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Stop if no improvement after 5 epochs
early_stopping_callback = EarlyStopping(patience=5, restore_best_weights=True)

# Reduce the learning rate once learning stagnates
reduce_lr_callback = ReduceLROnPlateau(
    monitor = 'val_loss', 
    patience = 2, # Reduce if no improvement after 2 epochs
    min_lr = 1e-6, # Lower bound on the learning rate 
    factor = 0.5, # => new_lr = lr * factor
    verbose = 1
)

# Training

In [11]:
from tensorflow.keras.optimizers import Adam
LEARNING_RATE = 2e-4
EPOCHS = 100
model.compile(optimizer=Adam(LEARNING_RATE))

In [ ]:
%%time
history = model.fit(
    train_generator,
    validation_data = valid_generator,
    validation_steps = len(valid_generator),
    steps_per_epoch = len(train_generator),
    epochs = EPOCHS,
    callbacks = [reduce_lr_callback, early_stopping_callback],
    verbose = 1
).history

Epoch 1/100
20/20 [==============================] - 19s 579ms/step - loss: 7.3287 - val_loss: 35.8491
Epoch 2/100
20/20 [==============================] - 11s 527ms/step - loss: 4.0740 - val_loss: 23.8821
Epoch 3/100
20/20 [==============================] - 11s 527ms/step - loss: 3.4812 - val_loss: 13.6435
Epoch 4/100
20/20 [==============================] - 11s 547ms/step - loss: 3.1589 - val_loss: 11.7119
Epoch 5/100
